In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain
import methods as m
import pickle
import warnings
import plotly as py
import os

warnings.filterwarnings('ignore')

In [ ]:
def get_list_videos(vid_dir):
    f = []
    for (dirpath, dirnames, filenames) in os.walk(vid_dir):
        f.extend(filenames)
    return f


def run_openpose(vid_dir,coord_location, openpose_location):
    os.chdir(openpose_location)
    for video in get_list_videos(vid_dir):
        os.system(r'bin\OpenPoseDemo.exe --video "{0}\{1}" --write_json "{2}\{1}"'.format(vid_dir, video, coord_location))

        
def create_total_feature_df(coord_df, video_number, return_df):
    feature_df = to_feature_df(coord_df, video_number)
    if return_df is None:
        return_df = feature_df
        print(return_df)
    else:
        return_df = return_df.append(feature_df)
    return return_df


def to_feature_df(coord_df, video_number):
    """
    Gets a DataFrame of coordinates and turns this into features.
    In this case, the standard deviation of movement vertically. Extension to also horizontally can be easily made in case this helps for discovering speed.

    :param coord_df: A dataframe containing all relevant coördiantes observed in the video.

    :return features_df: returns a dataframe containing standard deviations of all observed coordinates
    """
    
    #Set video number
    coord_df['video'] = video_number
    
    #extract basic std deviation features of all joints
    feature_df = coord_df.pivot_table(index=['video', 'Fragment'], columns='Point', values='y', aggfunc=np.std)
  
    #set video index
    feature_df['video'] = feature_df.index

    #Add value representing how much (in absoluut values) someone leaned forward
    feature_df['Forward_leaning'] = m.forward_leaning_angle(coord_df)

    return feature_df

# def forward_leaning(coord_df):
#     """
#     Create forward leaning feature to be used in classification. The forward leaning feature discribes to what extent a person
#     leans forward. which could be an indicator of a good runner

#     :param coord_df:  A dataframe containing all relevant coördiantes observed in the video.
#     :return return_list: returns a list containing containing the absoluut distance that is leaned forward
#     """
#     fragments = set(coord_df.Fragment)
#     return_list = []

#     for i in range(len(fragments)):
#         fragment_df = coord_df[coord_df['Fragment'] == i+1]
#         shoulder_df = fragment_df[fragment_df['Point'] == 'Right Shoulder']
#         hip_df = fragment_df[fragment_df['Point'] == 'Right Hip']
#         frames = set(fragment_df.Frame)
#         temp_sum = 0
#         frame_count = 0
#         for j in range(len(frames)):
#             difference = shoulder_df.iloc[j, 3] - hip_df.iloc[j, 3]
#             #couldn't think of a smarter way to not take the nan values into account for the average
#             if difference > 1:
#                 frame_count += 1
#                 temp_sum += difference
#             if difference < -1:
#                 frame_count += 1
#                 temp_sum += difference
#         return_list.append(abs(temp_sum / frame_count))
#     return return_list

In [ ]:
def prepare_data_for_classification(vid_dir,coord_location, openpose_location):
    #run_openpose(vid_dir,coord_location, openpose_location)
    video_number = 1
    return_df = None
    for video in get_list_videos(vid_dir):
        image_h, image_w, fps = m.determine_video_meta_data(r'{}\{}'.format(vid_dir, video))

        
#        people_per_file = m.get_openpose_output(r'{}\{}'.format(coord_location,video))
        period_person_division = m.get_period_person_division(m.get_openpose_output(r'{}\{}'.format(coord_location,video)), fps)
        person_period_division = m.get_person_period_division(period_person_division)
        
        mean_x_per_person = m.get_mean_x_per_person(person_period_division)
        
        normalized_moved_distance_per_person = m.normalize_moved_distance_per_person(mean_x_per_person)
        maximum_normalized_distance = max(normalized_moved_distance_per_person.values())
        
        movement_threshold = maximum_normalized_distance / 4
        
        moving_people = [key for key, value in normalized_moved_distance_per_person.items() if value > movement_threshold]
        
        
        person_plottables_df = m.get_person_plottables_df(mean_x_per_person, moving_people)
        
        dbscan_subsets = m.get_dbscan_subsets(maximum_normalized_distance, person_plottables_df)
        max_dbscan_subset = dbscan_subsets[
            np.argmax([sum([len(person_period_division[person]) for person in subset]) for subset in dbscan_subsets])]
        
        plottable_people = m.determine_plottable_people(person_plottables_df, 
                                                      max_dbscan_subset, 
                                                      maximum_normalized_distance*4)
        running_fragments, turning_fragments, fragments = m.get_running_and_turning_fragments(plottable_people, 
                                                                                                    mean_x_per_person, 
                                                                                                    person_plottables_df, 
                                                                                                    moving_people)
        
      #  coord_list = m.prepare_data_for_plotting(period_person_division, plottable_people, running_fragments)
        coord_df = m.get_dataframe_from_coords(m.prepare_data_for_plotting(period_person_division, 
                                                                           plottable_people, 
                                                                           running_fragments))
        return_df = create_total_feature_df(coord_df, video_number, return_df)

        video_number +=1
    return return_df
    
    
        
# extract_features(r'F:\Dropbox\TUe\DS-E\2017-2018\JM0130 Data Entrepreneurship in Action II\4. 2D_pose_estimation\videos',
#              r'F:\Dropbox\TUe\DS-E\2017-2018\JM0130 Data Entrepreneurship in Action II\4. 2D_pose_estimation\coordinates',
#             r'C:\Users\Herbert van Leeuwen\Desktop\openpose-1.2.1-win64-binaries\openpose-1.2.1-win64-binaries')

prepare_data_for_classification(r'C:\Users\herbe\Dropbox\TUe\DS-E\2017-2018\JM0130 Data Entrepreneurship in Action II\4. 2D_pose_estimation\videos',
             r'C:\Users\herbe\Dropbox\TUe\DS-E\2017-2018\JM0130 Data Entrepreneurship in Action II\4. 2D_pose_estimation\coordinates',
            r'C:\Users\Herbert van Leeuwen\Desktop\openpose-1.2.1-win64-binaries\openpose-1.2.1-win64-binaries')